# Fetching Stock Prices

In [11]:
import pandas as pd
import yfinance as yf
import datetime
from joblib import Parallel, delayed
import multiprocessing
import json 
import json.decoder
import os 
import requests
import pandas_datareader

datetime.date(2023, 1, 13)

In [31]:
env_json = {
    'Current_day': 1
}
with open('env.json',"w") as outfile:
    json.dump(env_json,outfile)

In [46]:
tmp_stocks = pd.read_csv('nasdaq.csv')

In [49]:
symbols = list(tmp_stocks['Symbol'])

In [53]:
tmp_json = {}
counter = 0 
sub_count = 0 
for idx, i in enumerate(symbols):
    if "^" in i: 
        continue 
    if sub_count % 500 == 0:
        counter += 1 
        tmp_json[counter] = []
    tmp_json[counter].append(i)
    sub_count += 1 

In [54]:
len(tmp_json)

14

In [39]:
with open("stocks.json","w") as outfile:
    json.dump(tmp_json,outfile)

In [45]:
pandas_datareader.data.DataReader('AAIC' , 'av-intraday', start=datetime.datetime(2022,1,1),end=datetime.datetime(2023,1,14),api_key='QVEGLYRJUK0ABVMT')

,open,high,low,close,volume
2022-12-30 09:31:00,2.9000,2.9100,2.9000,2.9100,255
2022-12-30 09:32:00,2.9100,2.9100,2.9100,2.9100,1000
2022-12-30 09:34:00,2.9101,2.9101,2.9101,2.9101,1000
2022-12-30 09:35:00,2.9200,2.9200,2.9200,2.9200,517
2022-12-30 09:45:00,2.9200,2.9200,2.9200,2.9200,250
...,...,...,...,...,...
2023-01-12 15:56:00,3.0000,3.0000,3.0000,3.0000,1735
2023-01-12 15:57:00,3.0000,3.0000,3.0000,3.0000,643
2023-01-12 15:58:00,3.0000,3.0000,2.9998,2.9998,8280
2023-01-12 15:59:00,3.0100,3.0100,3.0100,3.0100,620


In [2]:
tmp = pd.read_csv("stocks.csv",sep=";")



In [3]:
countries_to_exclude = ["India","China","South Korea", 
                        "Hong Kong", "Malaysia", "Taiwan", 
                        "Indonesia", "Thailand", "Singapore",
                        "Argentina", "Brazil", "Venezuela",
                        "Israel", "Qatar", "Russia", "Turkey",
                        "Australia", "New Zealand"]
for i in countries_to_exclude:
    tmp = tmp[tmp["Country"] != i]


In [4]:
stock_list = tmp["Ticker"]
today = datetime.date.today()
last_week = today - datetime.timedelta(days=7)
two_years = today - datetime.timedelta(days=729)

In [9]:
def processInput(ticker,time):

    if os.path.isfile(f"finished_{time}.txt"):
        pass
    else:
        with open(f"finished_{time}.txt","w") as infile:
            infile.write("\n")
    with open(f"finished_{time}.txt","r") as infile:
        if ticker in infile.read().split("\n"):
            return
    try: 
        count = 0 
        # print(ticker)
        # for year in years:
        #     for month in range(len(months)):
                # if (year,months[month]) in hour_json[ticker]:
                #     continue
                # else:
        print(f"{ticker}")
        ticker_download = yf.download(tickers=f"{ticker}", start=f"{two_years}",end=f"{today}",interval=time,auto_adjust=True,threads=True)
        print(ticker_download)
        # tmp = ticker_download.index.to_list()[-1][:4]
        # print(tmp)
        # print(ticker_download)
        # if tmp == "2021":
        ticker_download.to_csv(f"./data/stock_data_{time}/{ticker}_data.csv",sep=";")
        with open("finished.txt","a") as finished_file:
            finished_file.write(f"{ticker}\n")
        # hour_json[ticker].append((year,months[month]))
    except requests.exceptions.ConnectionError:
        processInput(ticker)
    except json.decoder.JSONDecodeError:
        return 
    except ValueError:
        return 
    except IndexError:
        return 
    except: 
        return 

In [10]:
# Parallel(n_jobs=4)(delayed(processInput)(i,"1h") for i in stock_list)
# Parallel(n_jobs=4)(delayed(processInput)(i,"1m") for i in stock_list)
Parallel(n_jobs=4)(delayed(processInput)(i,"1d") for i in stock_list)


-H.V
-A.TO
-B.TO
-P.V
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- -H.V: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- -P.V: No timezone found, symbol may be delisted
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- -B.TO: No timezone found, symbol may be delisted
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- -A.TO: No timezone found, symbol may be delisted
-PA.TO
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
-PB.TO
-PC.TO
-PD.TO
[*********************100%***********************]  1 of 1 completed

1 Fail

KeyboardInterrupt: 